In [1]:
#Import packages
import matplotlib.pyplot as plt
from math import exp
from scipy import stats
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 300)

### Import required data

In [2]:
df = pd.read_csv('kc_house_data_test_features.csv')

In [3]:
import pickle

In [4]:
infile = open("model.pickle",'rb')
model = pickle.load(infile)
infile.close()

In [5]:
infile = open("other_info.pickle",'rb')
other_info = pickle.load(infile)
infile.close()

In [6]:
print(model.intercept_)
print(len(model.coef_))

-112589781.47518131
106


In [7]:
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [8]:
selected = other_info

### Update test set to be consistent with training set

In [9]:
#Replacing zero bedrooms based on number of floors 
avg_bed = df['bedrooms'].mean()/df['floors'].mean()
df['bedrooms'] = np.where(df['bedrooms']==0, df['floors']*avg_bed, df['bedrooms'])

In [10]:
#Replacing zero bathrooms based on number of floors 
avg_bath = df['bathrooms'].mean()/df['floors'].mean()
df['bathrooms'] = np.where(df['bathrooms']==0, df['floors']*avg_bed, df['bathrooms'])

In [11]:
#Created a dummy variable for zipcode
zip_df = df['zipcode']
dummy_df = pd.get_dummies(zip_df, prefix='var_', drop_first=True)

In [12]:
df = pd.concat([df, dummy_df], axis = 1)

In [13]:
#Creating a new list for year sold
years = []
for date in df['date']:
    years.append(date[0:4])
df['year_sold']=years

In [14]:
#Creating a new variable for age of building since sale
df['year_sold'] = pd.to_numeric(df['year_sold'])

In [15]:
df['age'] = df['year_sold']-df['yr_built']

In [16]:
#Create column for most recent time work was done (if not renovated, this is the year built)
df['most_recent']=np.where(df['yr_renovated']==0, df['yr_built'], df['yr_renovated'])

In [17]:
#Engineering a new feature for whether or not the house was renovated
df['renovated'] = np.where(df['yr_renovated']==0,0,1)

In [18]:
#Engineering a new feature for whether or not the house is north or east of the Space Needle
space_lat = 47.6
space_long = -122.3
df['n']=np.where(df['lat']>space_lat, 1, 0)
df['e']=np.where(df['long']>space_long, 1, 0)

In [19]:
#Engineering a new feature for abs value of difference in lat from space needle
df['lat_from'] = abs(df['lat']-space_lat)

In [20]:
#Engineering a new feature for abs value of difference in long from space needle
df['long_from'] = abs(df['long']-space_long)

In [21]:
#Adding polynomial features
df['sqft_living^2'] = df['sqft_living']*df['sqft_living']
df['sqft_lot^2'] = df['sqft_lot']*df['sqft_lot']
df['sqft_living15^2'] = df['sqft_living15']*df['sqft_living15']
df['sqft_lot15^2'] = df['sqft_lot15']*df['sqft_lot15']
df['lat_from^2'] = df['lat_from']*df['lat_from']
df['long_from^2'] = df['long_from']*df['long_from']

In [22]:
#Adding interaction features
df['sqft_living-n'] = df['sqft_living']*df['n']
df['sqft_lot-n'] = df['sqft_lot']*df['n']
df['sqft_living15-n'] = df['sqft_living15']*df['n']
df['sqft_lot15-n'] = df['sqft_lot15']*df['n']
df['sqft_living-e'] = df['sqft_living']*df['e']
df['sqft_lot-e'] = df['sqft_lot']*df['e']
df['sqft_living15-e'] = df['sqft_living15']*df['e']
df['sqft_lot15-e'] = df['sqft_lot15']*df['e']

### Get Predictions

In [23]:
#Previewing df with selected columns for model
df[selected]

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,lat,long,sqft_living15,sqft_lot15,year_sold,age,most_recent,renovated,n,e,lat_from,long_from,sqft_living^2,sqft_lot^2,sqft_living15^2,sqft_lot15^2,lat_from^2,long_from^2,sqft_living-n,sqft_lot-n,sqft_living15-n,sqft_lot15-n,sqft_living-e,sqft_lot-e,sqft_living15-e,sqft_lot15-e,var__98002,var__98003,var__98004,var__98005,var__98006,var__98007,var__98008,var__98010,var__98011,var__98014,var__98019,var__98022,var__98023,var__98024,var__98027,var__98028,var__98029,var__98030,var__98031,var__98032,var__98033,var__98034,var__98038,var__98039,var__98040,var__98042,var__98045,var__98052,var__98053,var__98055,var__98056,var__98058,var__98059,var__98065,var__98070,var__98072,var__98074,var__98075,var__98077,var__98092,var__98102,var__98103,var__98105,var__98106,var__98107,var__98108,var__98109,var__98112,var__98115,var__98116,var__98117,var__98118,var__98119,var__98122,var__98125,var__98126,var__98133,var__98136,var__98144,var__98146,var__98148,var__98155,var__98166,var__98168,var__98177,var__98178,var__98188,var__98198,var__98199
0,4.0,2.50,2270,11500,1.0,0,0,3,8,1540,730,47.7089,-122.241,2020,10918,2014,47,1967,0,1,1,0.1089,0.059,5152900,132250000,4080400,119202724,0.011859,0.003481,2270,11500,2020,10918,2270,11500,2020,10918,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4.0,2.50,2270,11500,1.0,0,0,3,8,1540,730,47.7089,-122.241,2020,10918,2015,48,1967,0,1,1,0.1089,0.059,5152900,132250000,4080400,119202724,0.011859,0.003481,2270,11500,2020,10918,2270,11500,2020,10918,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3.0,2.50,1470,1779,2.0,0,0,3,8,1160,310,47.5472,-121.998,1470,1576,2014,9,2005,0,0,1,0.0528,0.302,2160900,3164841,2160900,2483776,0.002788,0.091204,0,0,0,0,1470,1779,1470,1576,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3.0,1.75,1280,16200,1.0,0,0,3,8,1030,250,47.7427,-122.071,1160,10565,2014,38,1976,0,1,1,0.1427,0.229,1638400,262440000,1345600,111619225,0.020363,0.052441,1280,16200,1160,10565,1280,16200,1160,10565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.0,2.75,2830,8126,2.0,0,0,3,8,2830,0,47.4863,-122.140,2830,7916,2015,10,2005,0,0,1,0.1137,0.160,8008900,66031876,8008900,62663056,0.012928,0.025600,0,0,0,0,2830,8126,2830,7916,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,3.0,2.50,1530,1131,3.0,0,0,3,8,1530,0,47.6993,-122.346,1530,1509,2014,5,2009,0,1,0,0.0993,0.046,2340900,1279161,2340900,2277081,0.009860,0.002116,1530,1131,1530,1509,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4318,4.0,2.50,2310,5813,2.0,0,0,3,8,2310,0,47.5107,-122.362,1830,7200,2015,1,2014,0,0,0,0.0893,0.062,5336100,33790969,3348900,51840000,0.007974,0.003844,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4319,2.0,0.75,1020,1350,2.0,0,0,3,7,1020,0,47.5944,-122.299,1020,2007,2014,5,2009,0,0,1,0.0056,0.001,1040400,1822500,1040400,4028049,0.000031,0.000001,0,0,0

In [24]:
final_answers = model.predict(df[selected])

In [25]:
final_answers

array([564409.9896526 , 594044.78670356, 436911.85342611, ...,
       454018.0180721 , 489281.33699714, 453857.06089459])

In [26]:
df_final = pd.DataFrame(final_answers)

In [27]:
df_final.to_csv('housing_preds_james_hodgens.csv')